In [1]:
%load_ext autoreload
%autoreload 2

## Training Transformer

In [2]:
import os
while 'notebooks' in os.getcwd():
    os.chdir("..")

import numpy as np
import pandas as pd 
from src.utils import train_test_split, get_sample_weights, get_eval_set
from src.preprocessing import preprocess_data
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel
from src.preprocessing import TextDataset
import torch
from torch.utils.data import DataLoader, Dataset
from IPython.display import clear_output
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, roc_auc_score
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, LoggingHandler
import logging
from copy import deepcopy
from sklearn.decomposition import PCA
from huggingface_hub import notebook_login
from sklearn.ensemble import RandomForestClassifier
from peft import get_peft_model, LoraConfig, TaskType
from collections import defaultdict
import transformers
from peft import get_peft_model, LoraConfig, TaskType
import re
from bert_score import BERTScorer
import langid
from src.utils import aggregate_samples, evaluate_model, compute_class_weights, remove_hashtag_links, get_first_texts
from torch.optim.lr_scheduler import ReduceLROnPlateau

from transformers import BitsAndBytesConfig

tqdm.pandas()

In [3]:
train_data, test_data = train_test_split()


  0%|          | 0/16 [00:00<?, ?it/s]

100%|██████████| 16/16 [00:05<00:00,  2.71it/s]


In [4]:
# sample_text = "Threre was a goal, half time, kick-off, full time, penalty, red card, yellow card, or own goal"
# # Define batch size
# batch_size = 1024

# # Initialize lists to store scores
# precisions = []
# recalls = []
# f1_scores = []

# data = pd.concat(train_data.values())

# # Create a progress bar
# for i in tqdm(range(0, len(data), batch_size), desc="Scoring Batches"):
#     # Slice the batch
#     batch = data.iloc[i:i + batch_size]['Tweet'].tolist()
    
#     # Compute BERTScore for the batch
#     P, R, F1 = scorer.score(batch, [sample_text] * len(batch), )
    
#     # Append scores
#     precisions.extend(P.tolist())
#     recalls.extend(R.tolist())
#     f1_scores.extend(F1.tolist())

In [5]:
# all_df = pd.concat(train_data.values())
# all_df['bertscore'] = f1_scores

In [6]:
# train = all_df.groupby(["MatchID", "PeriodID"], as_index=False).apply(lambda x: x.sort_values("bertscore").iloc[-50:])

In [8]:
df = pd.concat(train_data)

tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096", cache_dir = '/Data')
df['tokens'] = df['Tweet'].progress_apply(tokenizer.tokenize)

target_words = [
    "goal", "penalty", "halftime", "full-time", "yellow", "red",
    "kickoff", "extra time", "stoppage time", "foul", "offside", "handball",
    "save", "tackle", "dribble", "corner", "substitution", "header",
    "free kick", "throw-in", "assist", "hat-trick", "own goal", "victory",
    "defeat", "draw", "win", "loss", "tie", "comeback", "goalkeeper",
    "striker", "midfielder", "defender", "referee", "fans", "var", "gooal"
]
target_words = set(tokenizer.tokenize(" ".join(target_words)))

def is_valid_text(t):
    for w in t:
        if w in target_words:
            return True
        
    return False

df['is_valid']= df['tokens'].progress_apply(is_valid_text)
# df['lan'] = df['Tweet'].progress_apply(lambda x : langid.classify(x)[0])

100%|██████████| 1472980/1472980 [00:01<00:00, 853498.06it/s]


In [9]:
en_df = df.query("is_valid == 1")#.query("lan == 'en' ")

In [10]:
en_df

ID  MatchID  PeriodID  EventType      Timestamp  \
0  14         0_0        0         0          0  1403725802000   
   29         0_0        0         0          0  1403725806000   
   36         0_0        0         0          0  1403725807000   
   41         0_0        0         0          0  1403725809000   
   76         0_0        0         0          0  1403725818000   
...           ...      ...       ...        ...            ...   
19 155326  19_129       19       129          1  1403560797000   
   155330  19_129       19       129          1  1403560798000   
   155448  19_129       19       129          1  1403560799000   
   155458  19_129       19       129          1  1403560799000   
   155507  19_129       19       129          1  1403560800000   

                                                       Tweet  \
0  14      Let's go Honduras, with a little help from Fra...   
   29      The more goals France scores today the less go...   
   36      Robbie Fowler predicting a Honduras win? #Worl...   
   41         #WorldCup2014 lets go Ecuador lets go Honduras   
   76      I’m following Honduras versus Switzerland in t...   
...                                                      ...   
19 155326  #mirelagh: #WorldCup2014/ #WM2014 / Full Time:...   
   155330            Great win can't wait for next game #MEX   
   155448  Round of 16 - 29 de Junio #MEX vs #NED #Brasil...   
   155458  Holland better not let up, El Tri is coming fo...   
   155507               We took over Twitter today lmao #MEX   

                                                      tokens  is_valid  
0  14      [Let, 's, Ġgo, ĠHonduras, ,, Ġwith, Ġa, Ġlittl...      True  
   29      [The, Ġmore, Ġgoals, ĠFrance, Ġscores, Ġtoday,...      True  
   36      [Rob, bie, ĠFowler, Ġpredicting, Ġa, ĠHonduras...      True  
   41      [#, World, C, up, 2014, Ġlets, Ġgo, ĠEcuador, ...      True  
   76      [I, âĢ, Ļ, m, Ġfollowing, ĠHonduras, Ġversus, ...      True  
...                                                      ...       ...  
19 155326  [#, mire, l, agh, :, Ġ#, World, C, up, 2014, /...      True  
   155330  [Great, Ġwin, Ġcan, 't, Ġwait, Ġfor, Ġnext, Ġg...      True  
   155448  [Round, Ġof, Ġ16, Ġ-, Ġ29, Ġde, ĠJun, io, Ġ#, ...      True  
   155458  [H, oll, and, Ġbetter, Ġnot, Ġlet, Ġup, ,, ĠEl...      True  
   155507  [We, Ġtook, Ġover, ĠTwitter, Ġtoday, Ġl, ma, o...      True  

[476071 rows x 8 columns]

In [94]:
possible_indices = set(train_data.keys())

test_indices = list(np.random.choice(list(possible_indices), size=3, replace = False,))
test_indices = [13,1,18]
all_train_indices = list(possible_indices.difference(set(test_indices)))
val_indices = [1,5,4,12,19]
# val_indices = list(np.random.choice(all_train_indices, 3, replace=False))
# train_indices = list(set(all_train_indices).difference(set(val_indices)))
train_indices = [0,3,2,7,11,]

test_indices = [14,17,13, 18]


train_df = aggregate_samples(en_df, train_indices, max_size = 10)
# train_df = get_samples(train_indices, df = train)
test_df = aggregate_samples(en_df, test_indices, max_size=10)
val_df = aggregate_samples(en_df, val_indices, max_size=10)

/users/eleves-a/2022/pedro.silva/Desktop/sub-event-detection/src/utils.py:152: FutureWarning: The provided callable <function mean at 0x7f90881d5ee0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  return (df.query(f"MatchID in {indices}")).groupby(["MatchID", "PeriodID"]).agg({
/users/eleves-a/2022/pedro.silva/Desktop/sub-event-detection/src/utils.py:152: FutureWarning: The provided callable <function mean at 0x7f90881d5ee0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  return (df.query(f"MatchID in {indices}")).groupby(["MatchID", "PeriodID"]).agg({
/users/eleves-a/2022/pedro.silva/Desktop/sub-event-detection/src/utils.py:152: FutureWarning: The provided callable <function mean at 0x7f90881d5ee0> is currently using SeriesGroupBy.mean.

In [95]:
train_df = remove_hashtag_links(train_df)
test_df = remove_hashtag_links(test_df)
val_df = remove_hashtag_links(val_df)

In [84]:
test_indices

[14, 17, 13, 18]

## Training the model

In [85]:
train_df['EventType'].value_counts(normalize=True)

EventType
1.0    0.550769
0.0    0.449231
Name: proportion, dtype: float64

In [86]:
val_df['EventType'].value_counts(normalize=True)

EventType
1.0    0.528158
0.0    0.471842
Name: proportion, dtype: float64

In [87]:
test_df['EventType'].value_counts(normalize=True)

EventType
1.0    0.551923
0.0    0.448077
Name: proportion, dtype: float64

In [48]:
from transformers import DataCollatorWithPadding

In [ ]:
# K Fold CV


tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096", cache_dir = '/Data')
# tokenizer.pad_token = tokenizer.eos_token


device = 'cuda'
final_results = []


train_dataset = TextDataset(
    train_df["Tweet"].tolist(), 
    train_df['ID'].tolist(),
    train_df["EventType"].tolist(), 
    tokenizer,
    train_df.index.get_level_values("MatchID").tolist()

)

val_dataset = TextDataset(
    val_df["Tweet"].tolist(), 
    val_df['ID'].tolist(),
    val_df["EventType"].tolist(), 
    tokenizer,
    val_df.index.get_level_values("MatchID").tolist()
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=data_collator)
val_dataloader = DataLoader(val_dataset, batch_size=8, collate_fn=data_collator)


model = AutoModelForSequenceClassification.from_pretrained("allenai/longformer-base-4096", cache_dir = '/Data', num_labels = 2,ignore_mismatched_sizes=True)
# for p in base_model.model.parameters():
#     p.requires_grad = False
model.to(device)

# model = get_peft_model(base_model, lora_config)
# for param in model.bert.parameters():
#     param.requires_grad = False

for name, param in model.longformer.named_parameters():
    if "layer.11" in name or "layer.10":  # Unfreeze last two layers
        param.requires_grad = True

    else:
        param.requires_grad = False
    
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2, verbose=True)


best_model = None
best_acc = -1

n_epochs = 10

labels = train_df["EventType"].tolist()

class_weights = compute_class_weights(train_df['EventType']).to(device)
# class_weights = torch.Tensor([0.6,0.4]).to(device)

# Define weighted loss function
loss_fn = torch.nn.CrossEntropyLoss(class_weights)

for epoch in range(n_epochs):
    all_preds = []
    all_labels = []
    epoch_loss = 0

    print(train_indices, val_indices)
    for i, batch in enumerate(tqdm(train_dataloader)):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.autocast( device_type = 'cuda'):
            # outputs = model(input_ids=input_ids, attention_mask=attention_mask, extra_feature = count)
            # loss = loss_fn(outputs, labels.squeeze() )

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels = labels)
            loss = loss_fn(outputs.logits, labels)
        loss.backward()

        
        optimizer.step()
        optimizer.zero_grad()

        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        epoch_loss += loss.item()

    acc = accuracy_score(all_labels, all_preds, )
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    conf_matrix = confusion_matrix(all_labels, all_preds)


    
    print(f"---------- Epoch {epoch} ------------")
    print(f"Training Loss : {epoch_loss}\n")
    print(f"Training Accuracy : {acc}\n")
    print(f"Training Precision : {precision}\n")
    print(conf_matrix)


    preds, labels = evaluate_model(val_df, val_dataloader, model)

    y_pred_val = pd.Series(preds, index=val_df.index)
    y_true_val = pd.Series(labels, index= val_df.index)
    combined_acc = pd.concat([y_pred_val, y_true_val], axis = 1)\
        .groupby("MatchID")\
        .apply(lambda x: accuracy_score(x[1], x[0]))


    print(combined_acc)
    scheduler.step(combined_acc.min())

    if combined_acc.min() > best_acc:
        best_acc = combined_acc.min()
        best_model = deepcopy(model)


# Combine results for this fold
# validation_results = pd.DataFrame({
#     "MatchID": validation_data["MatchID"].values,
#     "true_values": labels,
#     "predictions": preds,
# })

# final_results.append(validation_results)

In [92]:
best_acc

0.6538461538461539

In [96]:
test_dataset = TextDataset(
    test_df["Tweet"].tolist(), 
    test_df['ID'].tolist(),
    test_df["EventType"].tolist(), 
    tokenizer,
    test_df.index.get_level_values("MatchID").tolist()
)

test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)


preds, labels = evaluate_model(test_df, test_dataloader, best_model, extra_feature=False)

Validation Accuracy : 0.6307692307692307

Validation auc : 0.6267440295494311

[[137  96]
 [ 96 191]]


In [98]:
y_pred_val = pd.Series(preds, index=test_df.index)
y_true_val = pd.Series(labels, index= test_df.index)
combined_acc = pd.concat([y_pred_val, y_true_val], axis = 1)\
    .groupby("MatchID")\
    .apply(lambda x: accuracy_score(x[1], x[0]))


print(combined_acc)

MatchID
13    0.676923
14    0.515385
17    0.676923
18    0.653846
dtype: float64


### Generating predictions

In [93]:
total_test_df = get_eval_set().set_index(["MatchID", "PeriodID"])
test_df = preprocess_data(total_test_df)




  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:01<00:00,  3.36it/s]


In [73]:
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096", cache_dir = '/Data')
test_df['tokens'] = test_df['Tweet'].progress_apply(tokenizer.tokenize)

target_words = [
    "goal", "penalty", "halftime", "full-time", "yellow", "red",
    "kickoff", "extra time", "stoppage time", "foul", "offside", "handball",
    "save", "tackle", "dribble", "corner", "substitution", "header",
    "free kick", "throw-in", "assist", "hat-trick", "own goal", "victory",
    "defeat", "draw", "win", "loss", "tie", "comeback", "goalkeeper",
    "striker", "midfielder", "defender", "referee", "fans", "var", "gooal"
]
target_words = set(tokenizer.tokenize(" ".join(target_words)))

def is_valid_text(t):
    for w in t:
        if w in target_words:
            return True
        
    return False

test_df['is_valid']= test_df['tokens'].progress_apply(is_valid_text)

/users/eleves-a/2022/pedro.silva/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 362397/362397 [00:00<00:00, 719927.55it/s]


In [74]:
# test_df['language'] = test_df['lan'].apply(lambda x: x[0])
test_df['language'] = "en"

# test_df_en = test_df.query("language == 'en' ")
test_df_en = test_df.query("is_valid == 1")

In [75]:
processed_test_df = test_df_en.groupby(["MatchID", "PeriodID"]).agg({
    "Tweet":   lambda x: get_first_texts(x, max_size=15),
    "ID": len
})

processed_test_df = remove_hashtag_links(processed_test_df)

In [76]:
processed_test_df

Tweet   ID
MatchID PeriodID                                                        
6       0         Fascinated for this  match. This will tell us ...   82
        1         "In a few minutes  of  x ...Can't wait"....Waa...   85
        2         I want Germany to win. \n Germany Vs. Ghana ki...   79
        3         Let's go ! Hoping for Schweinsteiger to at lea...  110
        4         Come on black   for the win! Down those  wanke...  151
...                                                             ...  ...
16      125       T-minus 5 minutes until my morning productivit...  137
        126       Stat comparison between Serbia & Germany s int...  115
        127       Zum GlÃ¼ck hat wenigstens KEINER beim Tippspie...  116
        128       ç›´å‰äºˆæƒ³0-0ã¯å¤–ã‚Œã€‚ã‚»ãƒ«ãƒ“ã‚¢å„ªä½ã...  109
        129       Five hours till kick-off!!!!  \nCara os cara  ...   94

[516 rows x 2 columns]

In [77]:
test_dataset = TextDataset(
    processed_test_df["Tweet"].tolist(), 
    processed_test_df['ID'].tolist(), 
    None,
    tokenizer,
    [0] * 516
)

test_dataloader = DataLoader(test_dataset, batch_size=32, collate_fn=data_collator)

In [78]:
preds, labels, probas = evaluate_model(processed_test_df, test_dataloader, best_model, use_labels=False, return_proba=True)

  0%|          | 0/17 [00:00<?, ?it/s]

100%|██████████| 17/17 [00:12<00:00,  1.33it/s]


In [79]:
processed_test_df['EventType'] = preds

In [80]:
processed_test_df['EventType'].value_counts(normalize = True)

EventType
0    0.643411
1    0.356589
Name: proportion, dtype: float64

In [81]:
pd.merge(
    total_test_df,
    processed_test_df[["Tweet", "EventType"]],
    left_index=True,
    right_index=True

)[['EventType','ID']]\
    .drop_duplicates("ID")\
    .set_index("ID")\
    .to_csv("predictions_2.csv")

EventType
0    0.831609
1    0.168391
Name: proportion, dtype: float64